In [26]:
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from pathlib import Path
from tqdm import tqdm
import torch
import yaml
import gc
import os

output_dir = ''
path_to_model = '' # fine-tune from model
path_to_dataset = ''
checkpoint = 'Salesforce/codet5p-220m-bimodal'  # initialize from checkpoint
batch_size = 8
epochs = 1
device = "cuda" if torch.cuda.is_available() else "cpu"

In [27]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModel.from_pretrained(
    path_to_model if not path_to_model == '' else checkpoint,
    trust_remote_code=True).to(device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB. GPU 0 has a total capacty of 4.00 GiB of which 0 bytes is free. Of the allocated memory 3.16 GiB is allocated by PyTorch, and 313.35 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [28]:
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [29]:
dataset = load_dataset(
    "json",
    data_files=path_to_dataset
)["train"].with_format("torch")

In [30]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [31]:
torch.cuda.empty_cache()
gc.collect()

9778

In [11]:
model.train()
pba = tqdm(dataloader)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

for epoch in range(epochs):
    for step, batch in enumerate(pba):
        optimizer.zero_grad()

        for k, v in batch.items():
            batch[k] = v.squeeze(1).to(device)

        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        pba.set_description_str(f"Epoch: {epoch} Step: {step} Loss: {loss.item():.4f}")

    model.save_pretrained(os.path.join(output_dir, f'epoch-{epoch}'))

  0%|          | 0/12828 [00:07<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 4.00 GiB of which 0 bytes is free. Of the allocated memory 3.16 GiB is allocated by PyTorch, and 313.35 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [32]:
train_info = {
    "path_to_model": path_to_model,
    "path_to_dataset": path_to_dataset,
    "checkpoint": checkpoint,
    "batch_size": batch_size,
    "epochs": epochs,
    "dataset": {
        "rows": dataset.num_rows,
    }
}

In [33]:
with open(os.path.join(output_dir, "train_info.yml"), "w") as f:
    yaml.dump(train_info, f)